<a href="https://colab.research.google.com/github/changsoooo/kisra/blob/TR-5/KRX%ED%81%AC%EB%A1%A4%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install kora -q
# from kora.selenium import wd
from bs4 import BeautifulSoup
import requests, time
import pandas as pd
from google.colab import drive
from collections import defaultdict
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
params = {'mktId': 'ALL',
'trdDd': '20210118',
##Noah datetime여기서 지정하면 되요!
'share': '1',
'money': '1',
'csvxls_isNo': 'false',
'name': 'fileDown',
'url': 'dbms/MDC/STAT/standard/MDCSTAT01501'}
##url부분만 수정하면 된다....해당 코드 찾아서 수정하면 된다

In [ ]:
def pre_dataframe(data, keys, dictionary):
  for row in data[:-1]:
    temp = row.split(',')
    for idx, value in enumerate(temp):
      value = value.replace('"', '')
      if 4<=idx<=5 or 7<=idx:
        dictionary[keys[idx]].append(int(value))
        continue
      dictionary[keys[idx]].append(value)
  return dictionary

In [ ]:
download_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
gen_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'

response = requests.post(gen_url, data = params)
code = BeautifulSoup(response.text, 'html.parser').text
file = requests.post(download_url, data = {'code' : code}, headers = {'Referer' : gen_url})
file.encoding = 'euc-kr'

soup = BeautifulSoup(file.text, 'html.parser')
keys = soup.prettify().split('\n')[0]
data = soup.prettify().split('\n')[1:]
keys = keys.split(',')
dictionary = defaultdict(list)

dictionary = pre_dataframe(data, keys, dictionary)
df = pd.DataFrame.from_dict(dictionary)
df.to_csv('/content/gdrive/MyDrive/크롤링.csv')

In [ ]:
df

,종목코드,종목명,시장구분,소속부,종가,대비,등락률,시가,고가,저가,거래량,거래대금,시가총액,상장주식수
0,060310,3S,KOSDAQ,중견기업부,2180,-5,-0.23,2185,2220,2150,251311,550779945,97669473980,44802511
1,095570,AJ네트웍스,KOSPI,,4485,-25,-0.55,4525,4560,4430,77316,346903875,209997993075,46822295
2,006840,AK홀딩스,KOSPI,,25200,-1100,-4.18,25600,25800,24500,143803,3621032050,333838537200,13247561
3,054620,APS홀딩스,KOSDAQ,중견기업부,7090,80,1.14,7090,7370,7050,128606,917581630,144595026890,20394221
4,265520,AP시스템,KOSDAQ,우량기업부,24200,-950,-3.78,24950,25050,24100,311774,7620890500,350421493400,14480227
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2525,000547,흥국화재2우B,KOSPI,,19800,-900,-4.35,20500,20500,19800,1729,34770950,3041280000,153600
2526,000545,흥국화재우,KOSPI,,8140,-330,-3.90,8340,8340,7600,39393,313153050,6251520000,768000
2527,003280,흥아해운,KOSPI,,258,0,0.00,0,0,0,0,0,30117000222,116732559
2528,037440,희림,KOSDAQ,중견기업부,4230,265,6.68,3965,4445,3955,1201707,5075269520,58892069250,13922475
